In [1]:
import networkx as nx
import random
import pandas as pd

In [2]:
def simulate_diffusion(G, seed_set, activation_prob=0.1):
    active_nodes = set(seed_set)
    newly_active_nodes = set(seed_set)
    
    while newly_active_nodes:
        next_newly_active_nodes = set()
        for node in newly_active_nodes:
            neighbors = set(G.neighbors(node))
            for neighbor in neighbors - active_nodes:
                if random.random() < activation_prob:  # Probability of activation
                    next_newly_active_nodes.add(neighbor)
        active_nodes.update(next_newly_active_nodes)
        newly_active_nodes = next_newly_active_nodes
    
    return len(active_nodes)

In [3]:
def estimate_viral_spread(G, seed_set, num_simulations=10, activation_prob=0.1):
    total_spread = 0
    for _ in range(num_simulations):
        total_spread += simulate_diffusion(G, seed_set, activation_prob)
    return total_spread / num_simulations

In [4]:
def viral_marketing(G, k, num_simulations=10, activation_prob=0.1):
    seed_set = set()
    for _ in range(k):
        best_node = None
        best_spread = 0
        for node in G.nodes:
            if node not in seed_set:
                temp_set = seed_set | {node}
                spread = estimate_viral_spread(G, temp_set, num_simulations, activation_prob)
                if spread > best_spread:
                    best_spread = spread
                    best_node = node
        if best_node is not None:
            seed_set.add(best_node)
    return seed_set

In [5]:
# Create a sample graph
df = pd.read_csv('../../facebook_clean_data/tvshow_edges.csv')
# Create a sample graph
G = nx.from_pandas_edgelist(df, 'node_1', 'node_2')
# Run greedy viral marketing
k = 3  # Number of nodes to select
selected_nodes = viral_marketing(G, k)

print(f"Selected nodes for viral marketing: {selected_nodes}")

Selected nodes for viral marketing: {3369, 3011, 2412}


Functionality:

simulate_diffusion: Simulates the process of information spreading through the network. Given an initial set of "seed" nodes, the function models how information spreads to neighboring nodes with a specified activation probability (activation_prob). The process continues until no new nodes are activated, and the function returns the total number of activated nodes.

estimate_viral_spread: Estimates the average number of nodes influenced by the seed set. It runs multiple simulations (controlled by num_simulations) of the diffusion process to account for randomness and provide a robust estimate of the spread.

viral_marketing: A greedy algorithm that selects k nodes to maximize the estimated viral spread. It iteratively evaluates each node not in the current seed set, adding the node that results in the highest increase in the spread of information. This process repeats until k nodes are selected.

Performance:

Efficiency: The greedy approach is effective for viral marketing by incrementally choosing nodes that offer the maximum marginal gain in information spread. It balances between computational complexity and accuracy but can become computationally intensive, especially for large networks, due to multiple simulations for each candidate node.

Scalability: The algorithm performs well for moderately sized graphs. However, its computational cost increases with the size of the graph and the number of simulations (num_simulations), which could be a limitation for very large networks.

Strengths:

Effectiveness: The greedy approach is known for its practical effectiveness in influence maximization problems. It provides a reasonable approximation of the optimal solution by focusing on nodes that offer the highest additional spread.

Customizability: The algorithm allows flexibility through parameters such as activation_prob (the probability of activation) and num_simulations (the number of simulations), making it adaptable to different network structures and scenarios.

Limitations:

Computational Cost: The primary limitation is the computational expense of running simulations for each node. For large networks or high values of k and num_simulations, this could be time-consuming.

Simplistic Diffusion Model: The model assumes a uniform activation probability for all edges, which may not accurately reflect real-world scenarios where activation probabilities could vary.